In [53]:
# import of required libraries and modules
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit ,QuantumRegister
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation


In [54]:
# get an abstract quantum circuit from Qiskit's library of circuits 
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [55]:
# see https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit
def get_qc_characteristics(qc):
    depth = qc.depth()
    # determine the number of qubits in `qc` and assign it to `num_qubits`    
    num_qubits = qc.num_qubits 
    # determine the operations in `qc` and assign it to `ops`
    ops = qc.count_ops()
    # determine the number of n-qubit operations (with n larger than 1) in `qc` and assign it to `num_multi_qubit_ops`, 
    num_multi_qubit_ops = 0
    for i in dict(ops.items()):
        if i not in ['id','h','rx','ry','rz','sx','sy','sz']:
            num_multi_qubit_ops+=dict(ops.items())[i]

    return {"depth":depth, "num_qubits":num_qubits, "ops":ops, "num_multi_qubit_ops": num_multi_qubit_ops}    
get_qc_characteristics(qc)

{'depth': 1,
 'num_qubits': 10,
 'ops': OrderedDict([('QFT', 1)]),
 'num_multi_qubit_ops': 1}

In [56]:

grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [57]:
#quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])
    
print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [58]:
qc_dec = qc.decompose()
print(qc_dec.count_ops()['cp'])
qc_dec.draw(fold=-1)

45


┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [59]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [60]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                        ┌───┐┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
q_8 -> 0 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3.1355) ├─────────────────■───────────────────────■─────────────────────■──────────────────────────────────■───────────■───────────────────────────────────■────────────────────────────────■─────────────────────────────────────────■──────────────────────────────■───────────────────────────────■────────────■──────────────────────────────────────■─────────■─────────────────────────────────────■─────────────────────■─────────────────────────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        └─┬─┘└──────────┘└─┬─┘└──┬───┬───┘┌──┴────┴──┐└───┬───┬────┘┌─────────┐    ┌─┴─┐    ┌──────────┐   ┌─┴─┐     ┌──────────┐  │     ┌────┐   ┌────────────┐      │           │                                   │                                │                                         │                              │                               │            │                                      │         │                                     │                     │                                         │                                                                                                                                                                                                                                                                                                                                                                                                                                                             
q_7 -> 1 ─────────────────────────────────┼────────────────┼─────┤ X ├────┤ Rz(-π/8) ├────┤ X ├─────┤ Rz(π/8) ├────┤ X ├────┤ Rz(-π/4) ├───┤ X ├─────┤ Rz(3π/4) ├──┼─────┤ √X ├───┤ Rz(3.1293) ├──────┼───────────┼───────────────────────■───────────┼───────────────────────────■────┼────────■────────────────────────────────┼───────■──────────────────────┼──────────────────────■────────┼────────────┼────────■──────────────────■──────────┼─────────┼──────────────────■────────■─────────┼─────────────────────┼────────────■─────────■──────────────────┼─────────────────────────────────■─────────────■───────────────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    

In [61]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [62]:

# assigning a 10-qubit linear `CouplingMap` to the variable cm
cm = []
for i in  range(9):
    cm.append([i,i+1])
    cm.append([i+1,i])
cm=CouplingMap(cm)
print(cm)




# addidng the transpiled quantum circuit
backend =  GenericBackendV2(num_qubits, coupling_map=cm)
qc_routed=generate_preset_pass_manager(2, backend=backend).run(qc)
print(qc_routed)

[[0, 1], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 5], [5, 4], [5, 6], [6, 5], [6, 7], [7, 6], [7, 8], [8, 7], [8, 9], [9, 8]]
                                                                              »
q_5 -> 0 ─────────────────────────────────────────────────────────────────────»
                                                                     ┌───┐    »
q_7 -> 1 ────────────────────────────────────────────────────────────┤ X ├────»
         ┌─────────┐    ┌────┐  ┌──────────┐                         └─┬─┘    »
q_9 -> 2 ┤ Rz(π/2) ├────┤ √X ├──┤ Rz(7π/8) ├──■────────────────■───────■──────»
         └─────────┘    └────┘  └──────────┘┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐»
q_8 -> 3 ───────────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├»
                                            └───┘└──────────┘└───┘└──────────┘»
q_6 -> 4 ─────────────────────────────────────────────────────────────────────»
                                                       

In [63]:
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [64]:
pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(target=backend.target)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

pm_staged.layout += generate_embed_passmanager(cm)
print(pm_staged)

In [65]:



#  see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_staged.routing = PassManager(StarPreRouting())
qc_routed = pm_staged.run(qc)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 34
  Number of qubits: 10
  Operations: {'cp': 45, 'swap': 43, 'h': 10}
  Number of multi-qubit Operations: 88


q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                         │P(π/512)      │                                                                                                                                                                                                
q_1 -> 1 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────X──────■──────────────X────────────────■─────────────X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                    │P(π/256)     │                                      │P(π/256)     │                                                                                                                                                                 
q_2 -> 2 ──────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────X──────■─────────────X─────────────────■─────────────X──────■─────────────X────────────────■─────────────X──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                               │P(π/128)     │                                      │P(π/128)     │                                     │P(π/128)     │                                                                                                                                  
q_3 -> 3 ──────────────────────────────────────────────────────────────────────────────────■────────────X──────■─────────────X─────────────────■─────────────X──────■─────────────X─────────────────■────────────X──────■─────────────X───────────────■────────────X─────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                           │P(π/64)     │                                      │P(π/64)      │                                      │P(π/64)     │                                    │P(π/64)     │                                                                                                     
q_4 -> 4 ───────────────────────────────────────────────────────────────■────────────X─────■────────────X─────────────────■─────────────X──────■─────────────X─────────────────■─────────────X──────■────────────X─────────────────■────────────X─────■────────────X───────────────■────────────X────────────────────────────────────────────────────────────────────────
                                                                        │P(π/32)     │                                    │P(π/32)      │                                      │P(π/32)      │                                     │P(π/32)     │                                  │P(π/32)     │                                                                        
q_5 -> 5 ────────────────────────────────────────────■────────────X─────■────────────X───────────────■─────────────X──────■─────────────X─────────────────■─────────────X──────■─────────────X────────────────■─────────────X──────■────────────X───────────────■────────────X─────■────────────X────────■────────────X──────────────────────────────────────────────────
                  

In [66]:
grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [67]:



#  see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names, target=backend.target))
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

Quantum circuit characteristics
  Depth: 127
  Number of qubits: 10
  Operations: {'cx': 219, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 219


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ┌───┐    ┌─────────────┐    ┌───┐     ┌────────────┐    ┌───┐                   ┌───┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
q_0 -> 0 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/1024) ├────┤ X ├─────┤ Rz(π/1024) ├────┤ X ├─────────■─────────┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ┌───┐    ┌────────────┐    ┌───┐     ┌───────────┐    ┌───┐                   ┌───┐   ┌────────────┐    └─┬─┘    └─────────────┘    └─┬─┘     └────────────┘    └─┬─┘       ┌─┴─┐       └─┬─┘                          ┌───┐    ┌────────────┐    ┌───┐     ┌───────────┐    ┌───┐                   ┌───┐                                                                                                                                 

In [68]:
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...


Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [69]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 144
  Number of qubits: 10
  Operations: {'rz': 185, 'cx': 179, 'sx': 66, 'x': 2}
  Number of multi-qubit Operations: 181


In [70]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes,
    # so backend.run would return noisy simulation results
    return backend.run(qc.measure_all(inplace=False), shots=7 * 1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim = noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.3838
Qiskit transpiler fidelity 0.6115


In [71]:
pm_opt = StagedPassManager()
# Selectin pass managers and passin for these five stages 
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replacin the n-qubit QFT operation with its decomposition in two-qubit gates
pm_opt.init = generate_unroll_3q(target=backend.target)
# Initializing the layout stage with an empty pass manager
pm_opt.layout = PassManager()
#   see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_opt.layout += TrivialLayout(cm)

#   see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_opt.routing = PassManager(StarPreRouting())

#   see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_opt.translation =  PassManager(BasisTranslator(SessionEquivalenceLibrary,backend.operation_names))
pm_opt.translation += HighLevelSynthesis()

#   see https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
pm_opt.optimization = PassManager()
pm_opt.optimization += Collect2qBlocks()
pm_opt.optimization += ConsolidateBlocks()
pm_opt.optimization += UnitarySynthesis(basis_gates = backend.operation_names)
pm_opt.optimization += CommutativeCancellation()
# pm_opt.scheduling = None


pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [72]:
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [73]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 154
  Number of qubits: 10
  Operations: {'rz': 294, 'sx': 182, 'cx': 133, 'x': 15}
  Number of multi-qubit Operations: 148
Reduction in two-qubit gates compared to qiskit 18.232%!


┌──────────┐┌────┐ ┌────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───┐      ┌─────────┐                ┌───┐  ┌────────────┐    ┌───┐        ┌───┐   ┌───────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
q_0 -> 0 ┤ Rz(-π/4) ├┤ √X ├─┤ Rz(-π) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├──────┤ Rz(π/2) ├────────────────┤ X ├──┤ Rz(1.5677) ├────┤ X ├────────┤ X ├───┤ Rz(-3π/4) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         ├──────────┤├────┤ └────────┘                                                                                                                                                                                                                                                                                                                                                                       

In [74]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.3838
Qiskit transpiler fidelity 0.6115
Own optimized transpiler fidelity 0.8899


In [75]:
class GatesPerQubit(AnalysisPass):
    def _init_(self):
        super().__init__()
    def run(self, dag):
        property_set = {"one_q_op":{} ,"layout":0,"clbit_write_latency":0,"conditional_latency":0,"node_start_time":0,"two_q_op":{}}
        for node in dag.topological_op_nodes():
            if len(node.qargs)==1:
                qubit=node.qargs[0]
                if qubit not in property_set['one_q_op']:
                    property_set["one_q_op"][qubit]=0
                property_set["one_q_op"][qubit]+=1
            elif len(node.qargs)==2:
                for qubit in node.qargs:
                    if qubit not in property_set["two_q_op"]:
                        property_set["two_q_op"][qubit]=0
                    property_set["two_q_op"][qubit]+=1
        self.property_set=property_set
   #see https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.AnalysisPass for analysis pass
    pass

In [76]:
grade_lab1_ex6(GatesPerQubit)

Submitting your answer. Please wait...
Failed: 403 Client Error: Forbidden for url: https://auth.quantum.ibm.com/api/users/loginWithToken


In [77]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

single-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 3): 14, Qubit(QuantumRegister(10, 'q'), 4): 15, Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 5): 4}
two-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 3): 14, Qubit(QuantumRegister(10, 'q'), 4): 11, Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 5): 3}


global phase: 2π
                                                               ┌───┐    ┌──────────┐   ┌───┐   ┌─────────┐                      ┌───┐ ┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌──────────┐                                                                                                                                                    
q_1 -> 2 ──────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/8) ├───┤ X ├───┤ Rz(π/8) ├──────────────────────┤ X ├─┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3π/4) ├───────────────────────────────────────────────────────────────────────────────────────────────────■────────────────■───────────────────────────────
         ┌─────────┐┌────┐┌──────────┐                         └─┬─┘    └──────────┘   └─┬─┘   └──┬───┬──┘     ┌───┐┌─────────┐ └─┬─┘ └──────────┘└─┬─┘└──┬────┬──┘┌──┴────┴──┐└──────────┘                                              ┌────┐  ┌───────┐        ┌────┐        ┌──────────┐┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐┌────┐┌─────────┐
q_3 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(7π/8) ├──■────────────────■───────■───────────────────────■────────┤ X ├─────■──┤ X ├┤ Rz(π/8) ├───■─────────────────■─────┤ √X ├───┤ Rz(-π/2) ├────────────────────────────────────────────────────■─────┤ √X ├──┤ Rz(π) ├──■─────┤ √X ├─────■──┤ Rz(3π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
         └─────────┘└────┘└──────────┘┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐   ┌────┐   ┌─────────┐   └─┬─┘   ┌─┴─┐└─┬─┘├─────────┴┐┌────┐┌──────────┐        ├────┤   └┬───────┬─┘                ┌────┐        ┌────────────┐┌────┐┌─┴─┐┌──┴────┴─┐└───────┘┌─┴─┐┌──┴────┴──┐┌─┴─┐├─────────┬┘└───┘└──────────┘└───┘└──────────┘└────┘└─────────┘
q_2 -> 4 ─────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(π/2) ├─────■─────┤ X ├──■──┤ Rz(π/16) ├┤ √X ├┤ Rz(-π/2) ├──■─────┤ √X ├────┤ Rz(π) ├───────■──────────┤ √X ├─────■──┤ Rz(-3π/16) ├┤ √X ├┤ X ├┤ Rz(π/2) ├─────────┤ X ├┤ Rz(3π/8) ├┤ X ├┤ Rz(π/4) ├────────────────────────────────────────────────────
          ┌────────┐┌────┐            └───┘└──────────┘└───┘└──────────┘   └────┘   └─────────┘           └───┘     └──────────┘└────┘└──────────┘┌─┴─┐┌──┴────┴─┐  └───────┘     ┌─┴─┐    ┌───┴────┴──┐┌─┴─┐└────────────┘└────┘└───┘└─────────┘         └───┘└──────────┘└───┘└─────────┘                                                    
q_0 -> 5 ─┤ Rz(-π) ├┤ √X ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(π/2) ├────────────────┤ X ├────┤ Rz(7π/16) ├┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          └────────┘└────┘                                                                                                                        └───┘└─────────┘                └───┘    └───────────┘└───┘

In [98]:
pg = Gate('Peres', 3, params=[], label='PG')
print(pg)

Instruction(name='Peres', num_qubits=3, num_clbits=0, params=[])


In [99]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [100]:
def get_qc_in(nq):
    # QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    # adding QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # performing swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    # adding circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # performing swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # adding inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [106]:
class PeresGateTranslation(TransformationPass):
    def _init_(self):
        super().__init__()
    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec

    def run(self, dag):
        property_set = {"one_q_op":{}
,"layout":0,"clbit_write_latency":0,"conditional_latency":0,"node_start_time":0,"two_q_op":{}}
        for node in dag.topological_op_nodes():
            if len(node.qargs)==1:
                qubit=node.qargs[0]
                if qubit not in property_set['one_q_op']:
                    property_set["one_q_op"][qubit]=0
                property_set["one_q_op"][qubit]+=1
            elif len(node.qargs)==2:
                for qubit in node.qargs:
                    if qubit not in property_set["two_q_op"]:
                        property_set["two_q_op"][qubit]=0
                    property_set["two_q_op"][qubit]+=1
        self.property_set=property_set
    def call(self, qc):
        self.run(circuit_to_dag(qc))
    def run(self, dag):
        for node in dag.op_nodes():
            if node.name=='Peres':
                dag.substitute_node_with_dag(node, circuit_to_dag(self.get_peres_decomposition()))
        return dag
  
    pass
  

In [107]:
grade_lab1_ex7(PeresGateTranslation, pg)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.
